# **Modelado: Volatilidad**

In [1]:
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
btc = pd.read_csv(r'https://raw.githubusercontent.com/TawnyVTC/Proyectos_UN/refs/heads/main/2025/Deep_Learning/Data/btc_1d_with_volatility_and_lags.csv')
btc['Open time'] = pd.to_datetime(btc['Open time'], format='%Y-%m-%d')


## **Split Temporal y Validación Cruzada**


En esta sección se replica la misma lógica empleada para la predicción del precio de cierre, pero enfocada en la volatilidad del Bitcoin. Primero, se generan variables retardadas de la volatilidad (lags de 7, 14, 21 y 28 días), que capturan la dependencia temporal del indicador y permiten al modelo aprender patrones históricos. Posteriormente, se crean los objetivos futuros correspondientes a los próximos siete días (t+1 a t+7), de modo que el modelo pueda realizar predicciones multihorizonte.

Las características de entrada incluyen tanto variables del precio (Close, LogReturn) como las medidas de volatilidad actual y retardada, buscando combinar información de nivel y dispersión. Finalmente, se eliminan los valores faltantes generados por los desplazamientos y se construyen las matrices de entrada (X) y salida (y) que alimentarán el flujo principal de entrenamiento, conformando el objeto timeSeries con el que se aplicarán los mismos experimentos de evaluación por número de lags.

In [3]:
# -----------------------
# 1. Crear lags de volatilidad
# -----------------------
for lag in [7, 14, 21, 28]:
    btc[f'Volatility_lag_{lag}'] = btc['Volatility'].shift(lag)

# -----------------------
# 2. Crear targets futuros (predicciones de volatilidad)
# -----------------------
for i in range(1, 8):  # t+1 a t+7
    btc[f'target_vol_t+{i}'] = btc['Volatility'].shift(-i)

# -----------------------
# 3. Definir features y targets
# -----------------------
features = [
    'Close', 'LogReturn', 'Volatility',
    'Volatility_lag_7', 'Volatility_lag_14', 'Volatility_lag_21', 'Volatility_lag_28'
]

targets = [f'target_vol_t+{i}' for i in range(1, 8)]

# -----------------------
# 4. Eliminar filas con NaN (por los shifts)
# -----------------------
btc = btc.dropna(subset=features + targets).reset_index(drop=True)

# -----------------------
# 5. Definir matrices de entrada y salida
# -----------------------
X = btc[features].values
y = btc[targets].values
dates = btc["Open time"].values

# -----------------------
# 6. Construcción del objeto timeSeries
# -----------------------
timeSeries = np.concatenate([X, y], axis=1)


In [4]:
from tsxv.splitTrainValTest import split_train_val_test_groupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

def _ensure_2d(arr):
    """
    Convierte el array a formato 2D.
    Si llega 1D → (n,1); si llega 3D → (n, timesteps*feats)
    """
    arr = np.asarray(arr)
    if arr.ndim == 1:
        return arr.reshape(-1, 1)
    if arr.ndim == 2:
        return arr
    if arr.ndim == 3:
        n, a, b = arr.shape
        return arr.reshape(n, a * b)
    raise ValueError(f"Array con ndim={arr.ndim} no soportado por esta función.")


def split_and_scale(timeSeries, n_steps_input=7, n_steps_forecast=7, n_steps_jump=1, target_col=2):
    X_list, y_list, Xcv_list, ycv_list, Xtest_list, ytest_list = split_train_val_test_groupKFold(
        timeSeries,
        n_steps_input,
        n_steps_forecast,
        n_steps_jump
    )

    X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
    y_train_scaled, y_val_scaled, y_test_scaled = [], [], []
    scalers_x, scalers_y = [], []

    for fold in range(len(X_list)):
        X_train_raw = _ensure_2d(X_list[fold])
        X_val_raw   = _ensure_2d(Xcv_list[fold])
        X_test_raw  = _ensure_2d(Xtest_list[fold])

        # 👇 Aquí filtramos solo la variable objetivo (columna target)
        y_train_raw = _ensure_2d(y_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_val_raw   = _ensure_2d(ycv_list[fold])[:, target_col:target_col + n_steps_forecast]
        y_test_raw  = _ensure_2d(ytest_list[fold])[:, target_col:target_col + n_steps_forecast]

        # Escaladores
        scaler_x = StandardScaler().fit(X_train_raw)
        scaler_y = StandardScaler().fit(y_train_raw)

        X_train_scaled.append(scaler_x.transform(X_train_raw))
        X_val_scaled.append(scaler_x.transform(X_val_raw))
        X_test_scaled.append(scaler_x.transform(X_test_raw))
        y_train_scaled.append(scaler_y.transform(y_train_raw))
        y_val_scaled.append(scaler_y.transform(y_val_raw))
        y_test_scaled.append(scaler_y.transform(y_test_raw))

        scalers_x.append(scaler_x)
        scalers_y.append(scaler_y)

        print(f"Fold {fold+1}: train {X_train_raw.shape} -> val {X_val_raw.shape} -> test {X_test_raw.shape}")

    return {
        'X_train': X_train_scaled,
        'X_val': X_val_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train_scaled,
        'y_val': y_val_scaled,
        'y_test': y_test_scaled,
        'scalers_x': scalers_x,
        'scalers_y': scalers_y
    }


## **Modelado con Deep Learning**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.stats.diagnostic import acorr_ljungbox
import pandas as pd
import numpy as np

In [6]:
def test_independencia_residuos(residuos, lags=10):
    resultado = acorr_ljungbox(residuos, lags=[lags], return_df=True)
    return resultado['lb_pvalue'].iloc[0]

def _calc_metrics_per_horizon(y_true, y_pred):
    """
    y_true, y_pred: arrays (n_samples, n_horizons)
    Devuelve DataFrame con métricas por horizonte.
    """
    n_outputs = y_true.shape[1]
    rows = []
    for h in range(n_outputs):
        yt = y_true[:, h]
        yp = y_pred[:, h]
        mae = mean_absolute_error(yt, yp)
        mse = mean_squared_error(yt, yp)
        rmse = np.sqrt(mse)
        # MAPE con epsilon para evitar división por cero
        mape = np.mean(np.abs((yt - yp) / (np.abs(yt) + 1e-8))) * 100
        rows.append({'Horizonte': h+1, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'MAPE': mape})
    df = pd.DataFrame(rows)
    return df

In [7]:
def entrenar_mlp_folds(data_dict, n_outputs=7, epochs=100, batch_size=32, verbose_fit=0):
    """
    Versión extendida:
    - Devuelve: resultados (lista), tablas_folds (lista de df_metrics por fold)
    - Cada df_metrics tiene métricas por horizonte + fila 'Promedio'
    """
    resultados = []
    tablas_folds = []

    for fold in range(len(data_dict['X_train'])):
        print(f"\n===== Fold {fold+1}/{len(data_dict['X_train'])} =====")

        X_train, X_val, X_test = data_dict['X_train'][fold], data_dict['X_val'][fold], data_dict['X_test'][fold]
        y_train, y_val, y_test = data_dict['y_train'][fold], data_dict['y_val'][fold], data_dict['y_test'][fold]
        scaler_y = data_dict['scalers_y'][fold]

        # --- Modelo ---
        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(n_outputs)
        ])
        model.compile(optimizer='adam', loss='mse')

        es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=epochs, batch_size=batch_size, verbose=verbose_fit, callbacks=[es])

        # --- Predicciones (desescaladas) ---
        yhat_train = scaler_y.inverse_transform(model.predict(X_train))
        yhat_val = scaler_y.inverse_transform(model.predict(X_val))
        yhat_test = scaler_y.inverse_transform(model.predict(X_test))
        ytrain_real = scaler_y.inverse_transform(y_train)
        yval_real = scaler_y.inverse_transform(y_val)
        ytest_real = scaler_y.inverse_transform(y_test)

        # --- Métricas por horizonte (test) ---
        df_metrics = _calc_metrics_per_horizon(ytest_real, yhat_test)
        # añadir fila promedio
        promedio = df_metrics.mean(numeric_only=True).to_dict()
        promedio['Horizonte'] = 'Promedio'
        df_metrics = pd.concat([df_metrics, pd.DataFrame([promedio])], ignore_index=True)

        # p-value BDS en residuos del horizonte 1 (h=1)
        resid_h1 = ytest_real[:, 0] - yhat_test[:, 0]
        try:
            pval = test_independencia_residuos(resid_h1, lags=10)
        except Exception as e:
            pval = np.nan

        # agregamos columna BDS_pvalue_h1 (solo en la primera fila será pval)
        bds_col = [np.nan] * len(df_metrics)
        if len(df_metrics) > 0:
            bds_col[0] = pval
        df_metrics['BDS_pvalue_h1'] = bds_col

        # print resumen por fold
        print(f"Fold {fold+1} | RMSE prom: {df_metrics.loc[df_metrics['Horizonte']=='Promedio','RMSE'].values[0]:.4f} | pval(h1): {pval}")

        resultados.append({
            'fold': fold+1,
            'rmse_test_prom': df_metrics.loc[df_metrics['Horizonte']=='Promedio','RMSE'].values[0],
            'pval_h1': pval,
            'y_train_real': ytrain_real, 'y_train_pred': yhat_train,
            'y_val_real': yval_real, 'y_val_pred': yhat_val,
            'y_test_real': ytest_real, 'y_test_pred': yhat_test,
            'df_metrics': df_metrics
        })

        tablas_folds.append(df_metrics)

    return resultados, tablas_folds

In [8]:
# ------------------------------------------------------------------
# Funciones para agregar resultados por lag y crear tablas resumen
# ------------------------------------------------------------------

def resumen_por_lag(tablas_folds):
    """
    tablas_folds: lista de dataframes (uno por fold), cada df tiene las filas por horizonte + 'Promedio'.
    Devuelve df resumen por horizonte con mean y std entre folds.
    """
    # extraer solo filas de horizonte 1..H (no 'Promedio') para hacer stats por horizonte
    # asumimos que todos los df tienen la misma cantidad de horizontes antes de la fila Promedio
    list_horizon_dfs = []
    for df in tablas_folds:
        # seleccionar solo filas cuyo 'Horizonte' no sea 'Promedio'
        df_h = df[df['Horizonte'] != 'Promedio'].copy()
        df_h = df_h.reset_index(drop=True)
        list_horizon_dfs.append(df_h)

    # concatenar en multiindex: fold, horizonte
    concat = pd.concat(list_horizon_dfs, keys=range(1, len(list_horizon_dfs)+1), names=['fold','row'])
    # Queremos agrupar por horizonte y calcular mean/std de métricas
    metrics = ['MAPE','MAE','RMSE','MSE']
    summary_rows = []
    for h in sorted(concat['Horizonte'].unique(), key=lambda x: int(x)):
        sel = concat[concat['Horizonte']==h]
        row = {'Horizonte': int(h)}
        for m in metrics:
            row[f'{m}_mean'] = sel[m].mean()
            row[f'{m}_std'] = sel[m].std()
        # p-value mean across folds: buscar pval en fold df (está en la primera fila BDS_pvalue_h1)
        # coletar pvals
        pvals = []
        for df in tablas_folds:
            pv = df['BDS_pvalue_h1'].iloc[0]
            if not np.isnan(pv):
                pvals.append(pv)
        row['BDS_pvalue_h1_mean'] = np.mean(pvals) if len(pvals)>0 else np.nan
        row['BDS_pvalue_h1_std'] = np.std(pvals) if len(pvals)>0 else np.nan

        summary_rows.append(row)

    df_summary = pd.DataFrame(summary_rows)
    return df_summary


In [9]:
# ------------------------------------------------------------------
# Función principal: evalúa lista de lags y devuelve todo lo necesario
# ------------------------------------------------------------------

def evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28], n_outputs=7,
                        epochs=80, batch_size=32, verbose_fit=0):
    """
    Ejecuta todo el flujo para cada n_lag:
    - split + scale
    - entrenar por folds
    - guarda tablas por fold y resumen por lag (mean + std)
    - devuelve estructura con todo para plotting y tablas
    """
    todos_lags = {}
    comparativa_resumen = []

    for n_lag in lags_list:
        print(f"\n=== Procesando n_lag = {n_lag} ===")
        data_dict = split_and_scale(timeSeries, n_steps_input=n_lag, n_steps_forecast=n_outputs)
        resultados, tablas_folds = entrenar_mlp_folds(data_dict, n_outputs=n_outputs,
                                                     epochs=epochs, batch_size=batch_size,
                                                     verbose_fit=verbose_fit)

        # df concatenado de métricas (todos los folds)
        df_metricas_comb = pd.concat([t.reset_index(drop=True) for t in tablas_folds], keys=range(1, len(tablas_folds)+1),
                                      names=['fold','row']).reset_index()
        # resumen por horizonte (mean, std)
        df_summary_h = resumen_por_lag(tablas_folds)

        # resumen global (media de RMSE, MAE, MAPE, MSE y pval promedio)
        rmse_prom = df_metricas_comb.groupby('Horizonte')['RMSE'].mean().mean()  # promedio de horizontes
        mae_prom  = df_metricas_comb.groupby('Horizonte')['MAE'].mean().mean()
        mape_prom = df_metricas_comb.groupby('Horizonte')['MAPE'].mean().mean()
        mse_prom  = df_metricas_comb.groupby('Horizonte')['MSE'].mean().mean()
        pvals = df_metricas_comb['BDS_pvalue_h1'].dropna()
        pval_prom = pvals.mean() if len(pvals)>0 else np.nan

        comparativa_resumen.append({
            'n_lag': n_lag,
            'RMSE_prom': rmse_prom,
            'RMSE_std': df_metricas_comb.groupby('Horizonte')['RMSE'].mean().std(),
            'MAE_prom': mae_prom,
            'MAPE_prom': mape_prom,
            'MSE_prom': mse_prom,
            'BDS_pvalue_h1_prom': pval_prom
        })

        todos_lags[n_lag] = {
            'resultados': resultados,         # lista de dict por fold (incluye df_metrics por fold)
            'tablas_folds': tablas_folds,     # lista de dataframes por fold
            'df_metricas_comb': df_metricas_comb,
            'df_summary_h': df_summary_h
        }

    df_comparativa = pd.DataFrame(comparativa_resumen).sort_values('n_lag').reset_index(drop=True)
    return todos_lags, df_comparativa

In [31]:
# ------------------------------------------------------------------
# Funciones de plotting
# ------------------------------------------------------------------

def escoger_folds_por_rmse(resultados):
    """
    Recibe lista de resultados (cada uno con 'rmse_test_prom').
    Devuelve indices best (min rmse), worst (max rmse) y median (por valor).
    """
    rmses = [r['rmse_test_prom'] for r in resultados]
    order = np.argsort(rmses)
    best_idx = int(order[0])
    worst_idx = int(order[-1])
    median_pos = int(len(order)//2)
    median_idx = int(order[median_pos])
    return best_idx, median_idx, worst_idx


def plot_series_fold(resultados_fold, fold_idx, n_steps_input, title_prefix=""):
    """
    Grafica las series reales y predichas para un fold específico.
    """
    y_train_real = resultados_fold['y_train_real']
    y_train_pred = resultados_fold['y_train_pred']
    y_val_real   = resultados_fold['y_val_real']
    y_val_pred   = resultados_fold['y_val_pred']
    y_test_real  = resultados_fold['y_test_real']
    y_test_pred  = resultados_fold['y_test_pred']

    fig, ax = plt.subplots(figsize=(10, 5))

    # Entrenamiento
    ax.plot(y_train_real[:, 0], label='Train Real', color='tab:blue', alpha=0.6)
    ax.plot(y_train_pred[:, 0], label='Train Predicho', color='tab:cyan', linestyle='--', alpha=0.8)

    # Validación
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_real)),
             y_val_real[:, 0], label='Val Real', color='tab:orange', alpha=0.6)
    ax.plot(range(len(y_train_real), len(y_train_real)+len(y_val_pred)),
             y_val_pred[:, 0], label='Val Predicho', color='tab:red', linestyle='--', alpha=0.8)

    # Test
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_real)),
             y_test_real[:, 0], label='Test Real', color='tab:green', alpha=0.6)
    ax.plot(range(len(y_train_real)+len(y_val_real),
                  len(y_train_real)+len(y_val_real)+len(y_test_pred)),
             y_test_pred[:, 0], label='Test Predicho', color='tab:purple', linestyle='--', alpha=0.8)

    ax.set_title(f"{title_prefix} | Fold {fold_idx+1} | Ventana = {n_steps_input}")
    ax.set_xlabel("Tiempo")
    ax.set_ylabel("Valor (Volatilidad o Precio)")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


def plot_rmse_bars(resultados, n_steps_input):
    """
    Grafica barras con el RMSE promedio por fold.
    """
    import numpy as np
    import matplotlib.pyplot as plt

    rmses = [r['rmse_test_prom'] for r in resultados]
    folds = np.arange(1, len(rmses)+1)

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(folds, rmses, color='skyblue', edgecolor='k')
    ax.set_title(f"RMSE por Fold | Ventana = {n_steps_input}")
    ax.set_xlabel("Fold")
    ax.set_ylabel("RMSE promedio (test)")
    ax.grid(axis='y', alpha=0.3)

    for i, v in enumerate(rmses):
        ax.text(folds[i], v, f"{v:.3f}", ha='center', va='bottom', fontsize=8)

    fig.tight_layout()
    return fig


def plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input):
    """
    Muestra el RMSE promedio (y std si existe) por horizonte.
    """
    fig, ax = plt.subplots(figsize=(8, 4))

    ax.plot(df_summary_h.index, df_summary_h['RMSE_mean'], marker='o', label='RMSE promedio')

    if 'RMSE_std' in df_summary_h.columns:
        ax.fill_between(df_summary_h.index,
                        df_summary_h['RMSE_mean'] - df_summary_h['RMSE_std'],
                        df_summary_h['RMSE_mean'] + df_summary_h['RMSE_std'],
                        color='blue', alpha=0.2, label='Desviación estándar')

    ax.set_title(f"RMSE promedio por horizonte | Ventana = {n_steps_input}")
    ax.set_xlabel("Horizonte de predicción (h)")
    ax.set_ylabel("RMSE")
    ax.legend()
    ax.grid(alpha=0.3)

    fig.tight_layout()
    return fig


In [11]:
# 1) Ejecutar evaluación completa
todos_lags, df_comparativa = evaluar_y_reportar(timeSeries, lags_list=[7,14,21,28],
                                                n_outputs=7, epochs=80, batch_size=32,
                                                verbose_fit=0)


=== Procesando n_lag = 7 ===
Fold 1: train (325, 98) -> val (108, 98) -> test (108, 98)
Fold 2: train (324, 98) -> val (108, 98) -> test (108, 98)
Fold 3: train (323, 98) -> val (108, 98) -> test (108, 98)
Fold 4: train (322, 98) -> val (108, 98) -> test (108, 98)
Fold 5: train (324, 98) -> val (108, 98) -> test (108, 98)

===== Fold 1/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Fold 1 | RMSE prom: 0.0608 | pval(h1): 0.9726225439211883

===== Fold 2/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Fold 2 | RMSE prom: 0.0728 | pval(h1): 0.0038120368733828964

===== Fold 3/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fold 3 | RMSE prom: 0.0657 | pval(h1): 0.12480855191333039

===== Fold 4/5 =====
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━

En la etapa de modelado de volatilidad, se evaluó el desempeño del modelo bajo diferentes configuraciones de retardos temporales (7, 14, 21 y 28 días). Los resultados muestran un comportamiento estable para horizontes cortos, con valores promedio de RMSE entre 0.06 y 0.07 para los modelos con 7 y 14 lags, indicando una adecuada capacidad del modelo para capturar la variabilidad de corto plazo en la serie.

A medida que se incrementa la ventana de rezagos (21 y 28 días), se observa un aumento gradual del error (RMSE entre 0.09 y 0.11), lo que sugiere una pérdida de precisión debido a la menor cantidad de datos efectivos para el entrenamiento y la posible dilución de patrones recientes. Los valores del p-valor del test BDS en la mayoría de los pliegues superan el umbral de significancia (p > 0.05), lo que sugiere que los residuales no presentan dependencias no lineales relevantes, respaldando la validez del modelo en términos de independencia temporal.

En general, el modelo logra una representación coherente del comportamiento dinámico de la volatilidad, especialmente en horizontes cortos, donde la memoria reciente del mercado parece ser más informativa para la predicción.

### *Tabla de Resumen Comparativo por Lags*

In [14]:
# 2) Ver tabla resumen comparativa de los 4 lags:
df_comparativa.to_csv(r'C:\DeepLearning\DL_Proyecto_2\data\volatilidad\tablas\resumen_comparativo.csv', index=False)
df_comparativa

,n_lag,RMSE_prom,RMSE_std,MAE_prom,MAPE_prom,MSE_prom,BDS_pvalue_h1_prom
0,7,0.065362,0.013288,0.041183,7.329915,0.004664,0.527326
1,14,0.065832,0.012181,0.044774,7.851903,0.004801,0.715526
2,21,0.106048,0.033549,0.067084,11.853536,0.013734,0.562758
3,28,0.091302,0.021816,0.067386,12.319823,0.009229,0.625582


Los resultados del modelo para la predicción de volatilidad del Bitcoin muestran un desempeño estable y coherente a través de las distintas configuraciones de retardos. Los menores errores promedio se observan con lags de 7 y 14 días, con valores de RMSE ≈ 0.065 y MAE ≈ 0.04–0.045, lo que evidencia una buena capacidad del modelo para capturar la dinámica de corto plazo.

En cambio, al incrementar la memoria temporal (lags de 21 y 28 días), los errores aumentan ligeramente (RMSE entre 0.09 y 0.11), lo que sugiere que incluir retardos más largos introduce ruido o patrones menos relevantes para la predicción inmediata.

El MAPE, que se mantiene entre 7% y 12%, refuerza la estabilidad general del modelo en términos relativos.
Finalmente, los valores del p-valor del test BDS (entre 0.52 y 0.71) superan el umbral de significancia (0.05), indicando que los residuales no presentan dependencias no lineales, por lo que el modelo logra capturar adecuadamente la estructura temporal de la volatilidad

In [32]:
import os

def resultados_lag_vol(n_lag):
    # 3) Para un lag específico:
    info = todos_lags[n_lag]
    resultados = info['resultados']   # lista por fold
    tablas_folds = info['tablas_folds']
    df_metricas_comb = info['df_metricas_comb']
    df_summary_h = info['df_summary_h']

    # Crear carpetas si no existen
    base_dir = fr'C:\DeepLearning\DL_Proyecto_2\data\volatilidad'
    tablas_dir = os.path.join(base_dir, 'tablas', f'lag_{n_lag}')
    figs_dir = os.path.join(base_dir, 'figs', f'lag_{n_lag}')
    os.makedirs(tablas_dir, exist_ok=True)
    os.makedirs(figs_dir, exist_ok=True)

    # 4) Guardar tablas por fold (test)
    for i, df in enumerate(tablas_folds):
        print(f"\n--- Fold {i+1} ---")
        df_path = os.path.join(tablas_dir, f'fold_{i+1}_lag_{n_lag}.csv')
        df.to_csv(df_path, index=False)
        print(df)

    # 5) Seleccionar best/median/worst y plotear
    best, med, worst = escoger_folds_por_rmse(resultados)

    # Mejor
    fig = plot_series_fold(resultados[best], best, n_steps_input=n_lag, title_prefix="Mejor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mejor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close()

    # Mediano
    fig = plot_series_fold(resultados[med], med, n_steps_input=n_lag, title_prefix="Mediano (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'mediano_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # Peor
    fig = plot_series_fold(resultados[worst], worst, n_steps_input=n_lag, title_prefix="Peor (RMSE)")
    fig.savefig(os.path.join(figs_dir, f'peor_rmse_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    # 6) Gráficas adicionales
    fig = plot_rmse_bars(resultados, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_por_fold_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    fig = plot_rmse_promedio_por_horizonte(df_summary_h, n_steps_input=n_lag)
    fig.savefig(os.path.join(figs_dir, f'rmse_promedio_por_horizonte_lag_{n_lag}.png'), bbox_inches='tight')
    plt.close(fig)

    print(f"\n✅ Resultados y gráficas guardados en: {figs_dir}")


### *Tablas de Metricas: Resumen por Horizonte y por Fold*

In [33]:
resultados_lag_vol(7)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE      MAPE  BDS_pvalue_h1
0         1  0.030249  0.007049  0.083957  5.779067       0.972623
1         2  0.043901  0.005275  0.072630  7.506183            NaN
2         3  0.034280  0.002521  0.050206  6.096272            NaN
3         4  0.045353  0.007061  0.084029  7.114623            NaN
4         5  0.039249  0.003304  0.057479  6.677889            NaN
5         6  0.023782  0.001292  0.035941  3.641690            NaN
6         7  0.024639  0.001716  0.041430  3.870203            NaN
7  Promedio  0.034493  0.004031  0.060810  5.812275            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.046059  0.005721  0.075637   8.910223       0.003812
1         2  0.041705  0.003771  0.061411   7.477346            NaN
2         3  0.050156  0.005986  0.077371   9.099414            NaN
3         4  0.050253  0.008036  0.089645   8.129569            NaN
4         5  0.056413  0.

In [34]:
resultados_lag_vol(14)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.037811  0.005012  0.070799   6.202642       0.844311
1         2  0.028294  0.001260  0.035497   5.244955            NaN
2         3  0.043389  0.003150  0.056129   7.719710            NaN
3         4  0.052653  0.003948  0.062832   8.981543            NaN
4         5  0.057342  0.006394  0.079962  10.725668            NaN
5         6  0.042420  0.005792  0.076102   7.349739            NaN
6         7  0.040714  0.003824  0.061842   6.822726            NaN
7  Promedio  0.043232  0.004197  0.063309   7.578141            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.042202  0.005545  0.074467   6.836543       0.896544
1         2  0.035416  0.002589  0.050885   5.541851            NaN
2         3  0.044371  0.002886  0.053723   7.813528            NaN
3         4  0.064968  0.007465  0.086399  10.686930            NaN
4         5  0.0

In [35]:
resultados_lag_vol(21)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.077957  0.032422  0.180062  12.593175       0.964089
1         2  0.067619  0.010725  0.103564  12.297044            NaN
2         3  0.053323  0.005750  0.075828  10.024552            NaN
3         4  0.077096  0.009398  0.096944  13.713713            NaN
4         5  0.056913  0.006028  0.077639  11.303682            NaN
5         6  0.077572  0.022261  0.149200  11.226348            NaN
6         7  0.076992  0.017898  0.133783  11.157223            NaN
7  Promedio  0.069639  0.014926  0.116717  11.759391            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.096223  0.048659  0.220589  14.411435       0.983753
1         2  0.071902  0.008267  0.090921  13.549644            NaN
2         3  0.074091  0.009713  0.098557  13.251196            NaN
3         4  0.080351  0.010989  0.104827  16.965065            NaN
4         5  0.0

In [36]:
resultados_lag_vol(28)


--- Fold 1 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.081712  0.009565  0.097798  14.655980       0.806478
1         2  0.051507  0.004923  0.070163   7.833900            NaN
2         3  0.054528  0.005173  0.071923  14.396486            NaN
3         4  0.045017  0.003361  0.057974   8.445345            NaN
4         5  0.079491  0.011951  0.109321  14.419879            NaN
5         6  0.059341  0.005023  0.070872  11.185952            NaN
6         7  0.070665  0.008345  0.091351  11.582158            NaN
7  Promedio  0.063180  0.006906  0.081343  11.788529            NaN

--- Fold 2 ---
  Horizonte       MAE       MSE      RMSE       MAPE  BDS_pvalue_h1
0         1  0.080551  0.012601  0.112255  14.961220       0.687233
1         2  0.055530  0.003958  0.062916   9.075311            NaN
2         3  0.064450  0.005783  0.076043  14.515606            NaN
3         4  0.049395  0.004821  0.069433   9.971223            NaN
4         5  0.0